In [4]:
import os
import numpy as np
import tensorflow as tf
import json
from PIL import Image
import matplotlib.pyplot as plt

# Load YOLO model
model_path = "./yolov8n_web_model/model.json"  # Adjust to your YOLO model path
yolo_model = TFSMLayer(model_path, call_endpoint='serving_default')

# Path to the test folder
test_folder = "./test"

# Confidence threshold for YOLO
threshold = 0.1

# Function to process a single image and tensor
def process_image_and_tensor(image_path, tensor_path=None):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB').resize((640, 640))
    image_array = np.array(image) / 255.0  # Normalize to [0, 1]
    input_tensor = tf.convert_to_tensor(image_array, dtype=tf.float32)
    input_tensor = tf.expand_dims(input_tensor, axis=0)  # Add batch dimension

    # Run inference with YOLO
    predictions = yolo_model.predict(input_tensor)

    # Post-process predictions
    detections = []
    for pred in predictions[0]:  # Assuming predictions[0] has [x, y, w, h, conf, class_probs...]
        x, y, w, h, conf = pred[:5]
        class_probs = pred[5:]
        class_id = np.argmax(class_probs)
        class_confidence = class_probs[class_id]

        # Filter detections by confidence
        if conf > threshold:
            detections.append({
                "box": [x, y, w, h],
                "confidence": conf,
                "class_id": class_id,
                "class_confidence": class_confidence
            })

    # Print detections
    print(f"\nDetections for {image_path}:")
    for det in detections:
        print(det)

    # Visualize detections
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    for det in detections:
        x, y, w, h = det['box']
        x_min, y_min = int(x - w / 2), int(y - h / 2)
        x_max, y_max = int(x + w / 2), int(y + h / 2)
        plt.gca().add_patch(plt.Rectangle((x_min, y_min), w, h, edgecolor='red', facecolor='none', linewidth=2))
        plt.text(x_min, y_min - 5, f"Class: {det['class_id']} ({det['confidence']:.2f})", color='red')
    plt.axis('off')
    plt.title(f"Detections for {os.path.basename(image_path)}")
    plt.show()

    # Compare with saved tensor if available
    if tensor_path:
        with open(tensor_path, 'r') as f:
            saved_tensor = np.array(json.load(f))
        print(f"Loaded saved tensor shape for {tensor_path}: {saved_tensor.shape}")

# Process all images and tensors in the folder
for file_name in os.listdir(test_folder):
    file_path = os.path.join(test_folder, file_name)

    # Check if the file is an image
    if file_name.endswith(".png") or file_name.endswith(".jpg"):
        # Find corresponding tensor file
        base_name = os.path.splitext(file_name)[0]
        tensor_file = os.path.join(test_folder, f"{base_name}.json")
        if os.path.exists(tensor_file):
            process_image_and_tensor(file_path, tensor_file)
        else:
            process_image_and_tensor(file_path)

ValueError: File format not supported: filepath=./yolov8n_web_model/model.json. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(./yolov8n_web_model/model.json, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).